# NLP Preprocessing

In [10]:
import numpy as np
import pandas as pd
import nltk
import data_cleaning as dc
import preprocess_NASDAQ_data as pre

453


FB-GOOGL-GOOG: No data found, symbol may be delisted
HRS: No data found, symbol may be delisted
INTC-USB: No data found, symbol may be delisted
AMZN-GPS: No data found, symbol may be delisted
TICKER: No data found, symbol may be delisted
CBS: No data found, symbol may be delisted
TWTR: No data found, symbol may be delisted
FB: No data found, symbol may be delisted
INFO: No data found, symbol may be delisted
JEC: No data found, symbol may be delisted
FNSR: No data found, symbol may be delisted
RE: No data found, symbol may be delisted
TMK: No data found, symbol may be delisted
NUAN: No data found, symbol may be delisted
TRQ: No data found, symbol may be delisted
DCIX: No data found, symbol may be delisted
FBHS: No data found, symbol may be delisted
PAH: No price data found, symbol may be delisted (period=10y)
WYN: No price data found, symbol may be delisted (period=10y)
LUK: No price data found, symbol may be delisted (period=10y)
FMSA: No price data found, symbol may be delisted (perio

In [11]:
NASDAQ_price = pre.result_df
NASDAQ_price.head(5)

,id,text,timestamp,source,symbols,company_names,url,verified,month,day,year,Price Day Before Tweet,Price Day of Tweet,Price Day After Tweet
0,1019696670777503700,VIDEO: “I was in my office. I was minding my o...,2018-07-18 21:33:26,GoldmanSachs,GS,The Goldman Sachs,https://twitter.com/i/web/status/1019696670777...,True,7,18,2018,202.197403,202.389938,200.980789
1,1019709091038548000,The price of lumber $LB_F is down 22% since hi...,2018-07-18 22:22:47,StockTwits,M,Macy's,https://twitter.com/i/web/status/1019709091038...,True,7,18,2018,29.254158,29.751337,30.74568
2,1019711413798035500,Who says the American Dream is dead? https://t...,2018-07-18 22:32:01,TheStreet,AIG,American,https://buff.ly/2L3kmc4,True,7,18,2018,46.698975,47.05748,45.964901
3,1019716662587740200,Barry Silbert is extremely optimistic on bitco...,2018-07-18 22:52:52,MarketWatch,BTC,Bitcoin,https://twitter.com/i/web/status/1019716662587...,True,7,18,2018,N/A,47.05748,45.964901
4,1019718460287389700,How satellites avoid attacks and space junk wh...,2018-07-18 23:00:01,Forbes,ORCL,Oracle,http://on.forbes.com/6013DqDDU,True,7,18,2018,44.699894,44.462223,44.279396


In [15]:
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob

# Load dataset
df = pd.read_csv('./stockerbot-export-preprocessed.csv', on_bad_lines='skip')

# Prepare the NLTK resources
nltk.download('wordnet')
nltk.download('stopwords')
lemmatizer = WordNetLemmatizer()

# Tracks if we did any preprocessing
preprocessed = False

# Add sentiment column with TextBlob if it doesn't exist
if 'tweet_polarity' not in df.columns:
    print('Calculating sentiment column...')
    df['tweet_polarity'] = df['text'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)
    preprocessed = True
if 'tweet_subjectivity' not in df.columns:
    print('Calculating subjectivity column...')
    df['tweet_subjectivity'] = df['text'].apply(lambda tweet: TextBlob(tweet).sentiment.subjectivity)
    preprocessed = True

# Apply preprocessing to the 'text' column if it doesn't exist
if 'preprocessed_tweet' not in df.columns:
    print('Preprocessing text column...')
    df['preprocessed_tweet'] = df['text'].apply(lambda tweet: dc.preprocess_tweet(tweet, lemmatizer))
    preprocessed = True
    
# INSERT STOCK DATA HERE
NASDAQ_price = pre.result_df
merged_df = pd.merge(NASDAQ_price, df, on='id', how='inner')
merged_df.to_csv('./merged-stock-data.csv')

display(merged_df.head(20))
    
# Save the preprocessed data
#if preprocessed:
    #print('Saving preprocessed data...')
    #df.to_csv('./stockerbot-export-preprocessed.csv', index=False)

# Display the preprocessed text
#pd.set_option('display.max_colwidth', None)
#display(df.head(20))

FileNotFoundError: [Errno 2] No such file or directory: './stockerbot-export-preprocessed.csv'

In [8]:
NASDAQ_price = pre.result_df
NASDAQ_price.head()

NameError: name 'pre' is not defined

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# TF-IDF vectorization for the 'preprocessed_tweet' column
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tfidf_features = tfidf_vectorizer.fit_transform(df['preprocessed_tweet'].astype('U'))  # Convert to Unicode

# One-hot encoding for categorical variables
onehot_encoder = OneHotEncoder()
onehot_features_source = onehot_encoder.fit_transform(df[['source']])
onehot_features_symbols = onehot_encoder.fit_transform(df[['symbols']])

# Scaling numerical features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df[['tweet_polarity', 'tweet_subjectivity']])

## Testing/Training Data

In [33]:
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split

# Combine all features into a single matrix
X = hstack([tfidf_features, onehot_features_source, onehot_features_symbols, scaled_features])

# Fill price_day_after with filler data
df['price_day_after'] = np.random.randint(0, 1001, size=len(df))

# The target variable
y = df['price_day_after'].values

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate

# Define the range of k values to test
k_values = range(1, 11)

# Initialize dictionaries to store the metrics
accuracy_scores = []
error_scores = []
precision_scores = []
recall_scores = []

# Perform cross-validation for each value of k
for k in k_values:
    # Create a kNN classifier with the current value of k
    knn = KNeighborsClassifier(n_neighbors=k)
    
    # Perform cross-validation and get the scores
    scores = cross_validate(knn, X_train, y_train.ravel(), cv=5, scoring=['accuracy', 'precision', 'recall'])
    
    # Calculate and store the metrics
    accuracy = np.mean(scores['test_accuracy'])
    error = 1 - accuracy
    precision = np.mean(scores['test_precision'])
    recall = np.mean(scores['test_recall'])
    
    accuracy_scores.append(accuracy)
    error_scores.append(error)
    precision_scores.append(precision)
    recall_scores.append(recall)
    
# Find the best k
best_k = k_values[np.argmax(accuracy_scores)]
print(f'Best k: {best_k}')

Best k: 1


In [34]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from scipy.sparse import hstack
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Create the models
linear_regression = LinearRegression()
logistic_regression = LogisticRegression(random_state=0, max_iter=1000)
lda = LinearDiscriminantAnalysis(shrinkage=None)
knn = KNeighborsClassifier(n_neighbors=best_k)
models = [linear_regression, logistic_regression, lda, knn]

# Create an instance of KFold
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Convert coo_matrix to csr_matrix for slicing
X_csr = X.tocsr()

# Perform k-fold validation
for train_index, test_index in kf.split(X):
    # Split the data into training and testing sets
    X_train, X_test = X_csr[train_index], X_csr[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Train your model and evaluate its performance
    for model in models:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
    
        # Print the mean squared error for each fold
        model_type = type(model).__name__
        print(f"{model_type} - Mean Squared Error: {mse}")

LinearRegression - Mean Squared Error: 104218.79264611927
LogisticRegression - Mean Squared Error: 166248.36564655934


TypeError: Sparse data was passed for X, but dense data is required. Use '.toarray()' to convert to a dense numpy array.